In [25]:
import pandas as pd
import numpy as np

### 1. Parsing Toronto postal codes table from the Wikipedia page using BeautifulSoup library

In [6]:
import requests
import urllib.request
import csv
from bs4 import BeautifulSoup

In [134]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)


In [135]:
soup = BeautifulSoup(response.text, 'html.parser')

In [136]:
#find the tag <table class="wikitable sortable">
table = soup.findAll('table',{"class":"wikitable sortable"})[0]
values =[]
tr = table.findAll(['tr'])

In [137]:
csvFile = open("toronto.csv",'wt',newline='',encoding='utf-8')
writer = csv.writer(csvFile)  
try:   
    for cell in tr:
        th = cell.find_all('th')
        th_data = [col.text.strip('\n') for col in th]
        td = cell.find_all('td')
        row = [i.text.replace('\n','') for i in td]
        writer.writerow(th_data+row)          
finally:   
    csvFile.close()

In [214]:
df_raw = pd.read_csv('toronto.csv')
df_raw.head()

Postal Code           Borough               Neighborhood
0         M1A      Not assigned                        NaN
1         M2A      Not assigned                        NaN
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront

### 2. Clean data

In [215]:
# Remove 'Not assigned' in Borough column
df_clean = df_raw[df_raw['Borough']!='Not assigned']
# Reindex
df_clean.reset_index(drop=True, inplace=True)

In [216]:
df_clean.isnull().values.any()
# Check 'Not assigned' in Neighborhood column
print("Number of rows with 'Not assigned' values: {}".format(len(df_clean[df_clean['Neighborhood']=='Not assigned'])))

Number of rows with 'Not assigned' values: 0


In [217]:
print('Shape of the frame: {}'.format(df_clean.shape))
df_clean

Shape of the frame: (103, 3)


Postal Code           Borough  \
0           M3A        North York   
1           M4A        North York   
2           M5A  Downtown Toronto   
3           M6A        North York   
4           M7A  Downtown Toronto   
..          ...               ...   
98          M8X         Etobicoke   
99          M4Y  Downtown Toronto   
100         M7Y      East Toronto   
101         M8Y         Etobicoke   
102         M8Z         Etobicoke   

                                          Neighborhood  
0                                            Parkwoods  
1                                     Victoria Village  
2                            Regent Park, Harbourfront  
3                     Lawrence Manor, Lawrence Heights  
4          Queen's Park, Ontario Provincial Government  
..                                                 ...  
98       The Kingsway, Montgomery Road, Old Mill North  
99                                Church and Wellesley  
100              Business reply mail Processing Centre  
101  Old Mill South, King's Mill Park, Sunnylea, Hu...  
102  Mimico NW, The Queensway West, South of Bloor,...  

[103 rows x 3 columns]

### 3. Obtain coordinates from the csv file and combine two dataframes

In [218]:
coord = pd.read_csv('Geospatial_Coordinates.csv')

#### Merge two dataframes by matching Postal Codes and keep only boroughs within Toronto 

In [219]:
df_clean = df_clean.merge(coord, left_on='Postal Code', right_on='Postal Code')
#df_trn
df_trn = df_clean[df_clean['Borough'].str.contains('Toronto')]
df_trn.reindex(columns=['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'])
df_trn.reset_index(drop=True, inplace=True)

In [220]:
df_trn

Postal Code           Borough  \
0          M5A  Downtown Toronto   
1          M7A  Downtown Toronto   
2          M5B  Downtown Toronto   
3          M5C  Downtown Toronto   
4          M4E      East Toronto   
5          M5E  Downtown Toronto   
6          M5G  Downtown Toronto   
7          M6G  Downtown Toronto   
8          M5H  Downtown Toronto   
9          M6H      West Toronto   
10         M5J  Downtown Toronto   
11         M6J      West Toronto   
12         M4K      East Toronto   
13         M5K  Downtown Toronto   
14         M6K      West Toronto   
15         M4L      East Toronto   
16         M5L  Downtown Toronto   
17         M4M      East Toronto   
18         M4N   Central Toronto   
19         M5N   Central Toronto   
20         M4P   Central Toronto   
21         M5P   Central Toronto   
22         M6P      West Toronto   
23         M4R   Central Toronto   
24         M5R   Central Toronto   
25         M6R      West Toronto   
26         M4S   Central Toronto   
27         M5S  Downtown Toronto   
28         M6S      West Toronto   
29         M4T   Central Toronto   
30         M5T  Downtown Toronto   
31         M4V   Central Toronto   
32         M5V  Downtown Toronto   
33         M4W  Downtown Toronto   
34         M5W  Downtown Toronto   
35         M4X  Downtown Toronto   
36         M5X  Downtown Toronto   
37         M4Y  Downtown Toronto   
38         M7Y      East Toronto   

                                         Neighborhood   Latitude  Longitude  
0                           Regent Park, Harbourfront  43.654260 -79.360636  
1         Queen's Park, Ontario Provincial Government  43.662301 -79.389494  
2                            Garden District, Ryerson  43.657162 -79.378937  
3                                      St. James Town  43.651494 -79.375418  
4                                         The Beaches  43.676357 -79.293031  
5                                         Berczy Park  43.644771 -79.373306  
6                                  Central Bay Street  43.657952 -79.387383  
7                                            Christie  43.669542 -79.422564  
8                            Richmond, Adelaide, King  43.650571 -79.384568  
9                        Dufferin, Dovercourt Village  43.669005 -79.442259  
10  Harbourfront East, Union Station, Toronto Islands  43.640816 -79.381752  
11                           Little Portugal, Trinity  43.647927 -79.419750  
12                       The Danforth West, Riverdale  43.679557 -79.352188  
13           Toronto Dominion Centre, Design Exchange  43.647177 -79.381576  
14       Brockton, Parkdale Village, Exhibition Place  43.636847 -79.428191  
15                     India Bazaar, The Beaches West  43.668999 -79.315572  
16                     Commerce Court, Victoria Hotel  43.648198 -79.379817  
17                                    Studio District  43.659526 -79.340923  
18                                      Lawrence Park  43.728020 -79.388790  
19                                           Roselawn  43.711695 -79.416936  
20                                   Davisville North  43.712751 -79.390197  
21                           Forest Hill North & West  43.696948 -79.411307  
22                      High Park, The Junction South  43.661608 -79.464763  
23                                 North Toronto West  43.715383 -79.405678  
24                The Annex, North Midtown, Yorkville  43.672710 -79.405678  
25                             Parkdale, Roncesvalles  43.648960 -79.456325  
26                                         Davisville  43.704324 -79.388790  
27                     University of Toronto, Harbord  43.662696 -79.400049  
28                                 Runnymede, Swansea  43.651571 -79.484450  
29                        Moore Park, Summerhill East  43.689574 -79.383160  
30          Kensington Market, Chinatown, Grange Park  43.653206 -79.400049  
31  Summerhill West, Rathnelly, South Hill, Forest...  43.686412 -